# Sequence Analysis

In [5]:
import pandas as pd
from msresist.pre_processing import preprocessing
from msresist.sequence_analysis import EM_CombinedClustering
from msresist.FileExporter import create_download_link
from sklearn.mixture import GaussianMixture
import warnings
warnings.simplefilter("ignore")

In [2]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 300)

ABC = preprocessing(motifs=True, Vfilter=True, FCfilter=True, log2T=True)

By setting 'GMMweight'=1.5 we already obtain exact cluster sizes as if we run GMM alone.

In [3]:
ncl, pYTS = 4, "Y"
clusters = EM_CombinedClustering(ABC, ncl, pYTS, GMMweight=1, max_n_iter=30)

,peptide-phosphosite,Master Protein Descriptions,PC9,Erl,R428,Erl/R428,Erl+HGF,Erl+FGF,Erl+IGF,KO Erl,KO R428,KO Erl/R428,position,cluster
8,AAAIAyGLDRT,Heat shock 70 kDa protein 1A,0.431886,0.894781,0.373969,-0.304444,-1.203818,-0.086804,-0.276191,-0.184870,-0.155607,0.511098,Y183-p,0
428,NMGGPyGGGNY,Heterogeneous nuclear ribonucleoproteins A2/B1,0.105260,-0.509831,-0.523870,-0.153921,0.539197,-0.903299,0.269387,0.368058,0.404884,0.404135,Y331-p,0
432,NQKHIyYITGE,Heat shock protein HSP 90-alpha,0.626567,0.214183,0.416046,-0.064293,0.178235,0.081767,0.095315,-0.458927,-0.516284,-0.572608,Y492-p,0
433,NRGPAyGLSRE,Transgelin-2,1.904443,0.544043,0.305980,-0.889446,0.132877,-0.472913,-0.063734,-0.449882,-0.236740,-0.774627,Y8-p,0
439,NYCPHyEKVSG,Ephrin-B1,0.988581,0.420986,0.190235,-0.537843,0.166722,-0.356496,-0.352347,-0.391797,0.181746,-0.309787,Y317-p,0
442,PAPLSyDSRPR,Tight junction protein ZO-1,0.486110,0.398106,-0.079906,-0.260941,-0.053860,-0.578136,-0.171009,-0.125943,0.107480,0.278099,Y1140-p,0
450,PETPGyVGFAN,Septin-2,1.259552,0.594600,0.140745,-0.667288,0.086001,-0.109128,-0.517450,-0.090551,-0.042932,-0.653549,Y17-p,0
452,PFGQGyTQPGP,Protein TFG,0.582781,0.191647,0.280545,-0.342809,-0.083625,-0.164704,-0.632819,0.060535,0.010030,0.098420,Y392-p,0
455,PGEVIyKCPKC,Palmitoyltransferase ZDHHC7,1.102630,-0.069657,0.318382,0.040563,-0.135237,-0.045763,-0.179829,-0.777063,0.391337,-0.645363,Y130-p,0
467,PLVGTyNTLLS,"Discoidin, CUB and LCCL domain-containing protein 2",0.631192,0.434815,0.046110,-0.638636,-0.174070,-0.181288,-0.097708,0.135375,0.018387,-0.174178,Y715-p,0


GMM cluster sizes: {0: 188, 2: 63, 3: 39, 1: 8}
0
1
2
3
4
5
6
convergence has been reached at iteration 6


Combined clustering results:

In [8]:
dictcl = pd.DataFrame(clusters).T
dictcl

,0,1,2,3
0,AAAIAYGLDRT,AVRLHYGLPVV,AANPAYGRYNP,AERDLYLENPE
1,AEEVEYYYRRA,AVRSAYGGPVG,AGEEHYNCISA,AVCSTYLQSRY
2,AHDSLYRVHMP,GSLPNYATING,AGMTGYGMPRQ,AWPSPYKDYEV
3,ASASEYAHRGL,GSRADYDTLSL,AIDVGYRHIDC,DKSREYDQLYE
4,ASEQGYEEMRA,PAPLSYDSRPR,AVGFEYQGKTE,ENSEIYDYLRQ
5,ASGIPYHSEVP,QKRWIYEDVER,DSFDDYPPRRG,GAEPNYHSLPS
6,ASHPNYPYSDE,TRGEGYAKPNE,EKQLLYSENKT,HVEAVYIDIAD
7,ATKVVYSAPRS,NSDFNYSYPTK,ELHRKYGTDLS,ILVGMYDKKPA
8,CKEAAYHPEVA,AKFINYVKNCF,ESYSVYVYKVL,KGGRGYDRDHV
9,DEEYIYMNKVT,PDRQAYEPPPP,FTSGKYQDVYV,NKHTNYTMEHI
